# Azure AI Search, Azure OpenAI, and QuotientAI RAG Pipeline

This pipeline combines Azure AI Search for information retrieval, Azure OpenAI for generation, and QuotientAI for evaluation.

In [ ]:
!pip install quotientai

In [27]:
import os
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os
import numpy as np
from typing import List, Dict, Any
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import (
    VectorizableTextQuery,
    VectorizedQuery
)
from quotientai import QuotientAI
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
from dataclasses import dataclass
from datetime import datetime
from dotenv import load_dotenv

# Azure Search
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

# OpenAI
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_API_KEY")
QUOTIENT_API_KEY = os.getenv("QUOTIENT_API_KEY")
os.environ["QUOTIENT_API_KEY"] = QUOTIENT_API_KEY

# Initialize Azure clients
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name="llamaindex-azure-aisearch-rag-literal-ai", credential=AzureKeyCredential(AZURE_SEARCH_KEY))
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-10-21",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)
# Initialize QuotientAI client
quotient_client = QuotientAI()


1. QuotientAI minimal resource classes and client

In [21]:
@dataclass
class ModelProvider:
    id: str
    name: str

@dataclass
class Model:
    id: str
    name: str
    provider: ModelProvider
    created_at: str

class ModelsResource:
    def __init__(self, client):
        self._client = client
    def list(self) -> List[Model]:
        # Placeholder
        return []

@dataclass
class DatasetRowMetadata:
    annotation: str = "ungraded"
    annotation_note: str = ""

@dataclass
class DatasetRow:
    id: str
    input: str
    context: str
    expected: str
    metadata: DatasetRowMetadata
    created_by: str
    created_at: datetime
    updated_at: datetime

@dataclass
class Dataset:
    id: str
    name: str
    created_by: str
    created_at: datetime
    updated_at: datetime
    description: str = ""
    rows: List[DatasetRow] = None

class DatasetsResource:
    def __init__(self, client):
        self._client = client

    def create(self, name: str, description: str = None, rows: List[dict] = None) -> Dataset:
        # Mock in-memory creation for demonstration
        now = datetime.utcnow()
        ds_id = "dataset-" + str(int(now.timestamp()))
        row_objs = []
        for i, r in enumerate(rows or []):
            row_objs.append(
                DatasetRow(
                    id=f"{ds_id}-{i}",
                    input=r.get("input", ""),
                    context=r.get("context", ""),
                    expected=r.get("expected", ""),
                    metadata=DatasetRowMetadata(
                        annotation=r["metadata"].get("annotation", "ungraded"),
                        annotation_note=r["metadata"].get("annotation_note", ""),
                    ),
                    created_at=now,
                    updated_at=now,
                    created_by="PlaceholderUser",
                )
            )
        return Dataset(
            id=ds_id,
            name=name,
            description=description or "",
            created_at=now,
            updated_at=now,
            created_by="PlaceholderUser",
            rows=row_objs,
        )

@dataclass
class Prompt:
    id: str
    name: str
    version: int
    system_prompt: str
    user_prompt: str
    created_at: datetime
    updated_at: datetime

class PromptsResource:
    def __init__(self, client):
        self._client = client

    def create(self, name: str, system_prompt: str = "", user_prompt: str = "") -> Prompt:
        now = datetime.utcnow()
        prompt_id = "prompt-" + str(int(now.timestamp()))
        return Prompt(
            id=prompt_id,
            name=name,
            version=1,
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            created_at=now,
            updated_at=now,
        )

class QuotientAI:
    def __init__(self):
        self.models = ModelsResource(self)
        self.datasets = DatasetsResource(self)
        self.prompts = PromptsResource(self)

    # Mock internal request methods
    def _get(self, path: str):
        pass
    def _post(self, path: str, data: dict):
        return data
    def _patch(self, path: str, data: dict):
        return data

## 3. Helper: Generate Embeddings with Azure OpenAI

In [22]:
def generate_embeddings(text: str):
    """
    Generate embeddings using Azure OpenAI.
    """
    embeddings_response = openai_client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME, 
        input=text
    )
    return embeddings_response.data[0].embedding


In [28]:
def call_chat_model(system_prompt: str, user_query: str) -> str:
    """
    Call the Azure OpenAI Chat Completion (GPT-4o or similar).
    You can also pass in context here if you prefer.
    """
    completion = openai_client.chat_completions.create(
        model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ],
    )
    return completion.choices[0].message.content

## 4. Vectorized Retrieval from Azure AI Search

In [23]:
def retrieve_documents(question: str, top_k: int = 3) -> List[Dict]:
    """
    Retrieve top_k documents from Azure Search using vector embeddings.
    """
    query_embedding = generate_embeddings(question)
    vector_query = VectorizedQuery(
        vector=query_embedding,
        k_nearest_neighbors=top_k,
        fields="embedding"  # your index vector field name
    )
    results = search_client.search(
        search_text=None,
        vector_queries=[vector_query],
        top=top_k
    )
    return list(results)

5. Example: Evaluate retrieval for multiple top_k values

In [24]:
test_query = "What is the annual deductible for individual coverage under the Northwind Health Plus plan?"

for k in [3, 10, 50]:
    print(f"\n==== Querying with top_k={k} ====")
    docs = query_azure_search(test_query, top_k=k)
    for i, doc in enumerate(docs):
        print(f"[Result #{i+1}]")
        print(f"Title: {doc.get('title', 'No title')}")
        print(f"Content snippet: {doc.get('content','')[:120]}...")
        print(f"Score: {doc.get('@search.score','No score')}")
        print("-" * 60)


==== Querying with top_k=3 ====
[Result #1]
Title: No title
Content snippet: ...
Score: 0.7765124
------------------------------------------------------------
[Result #2]
Title: No title
Content snippet: ...
Score: 0.7631142
------------------------------------------------------------
[Result #3]
Title: No title
Content snippet: ...
Score: 0.7506997
------------------------------------------------------------

==== Querying with top_k=10 ====
[Result #1]
Title: No title
Content snippet: ...
Score: 0.7765124
------------------------------------------------------------
[Result #2]
Title: No title
Content snippet: ...
Score: 0.7631142
------------------------------------------------------------
[Result #3]
Title: No title
Content snippet: ...
Score: 0.7506997
------------------------------------------------------------
[Result #4]
Title: No title
Content snippet: ...
Score: 0.7367365
------------------------------------------------------------
[Result #5]
Title: No title
Content snippet:

##  6. Create an Evaluation Dataset in Quotient

In [25]:
# Provided evaluation dataset
eval_dataset = [
    {
        "id": "Q001",
        "input": "What is the annual deductible for individual coverage under the Northwind Health Plus plan?",
        "context": "The Northwind Health Plus plan has a calendar year deductible of $1,500 for individuals for in-network services.",
        "expected": "$1,500",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Verified deductible from Northwind Health Plus benefits summary.",
        },
    },
    {
        "id": "Q002",
        "input": "Does the Northwind Standard plan cover out-of-network emergency services?",
        "context": "Northwind Standard does not offer coverage for emergency services, mental health and substance abuse coverage, or out-of-network services.",
        "expected": "No",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Aligned with coverage exclusions listed for the Northwind Standard plan.",
        },
    },
    {
        "id": "Q003",
        "input": "What fitness activities are covered under the PerksPlus program?",
        "context": "PerksPlus covers gym memberships, personal training sessions, yoga and Pilates classes, fitness equipment purchases, sports team fees, health retreats and spas, outdoor activities like rock climbing, and virtual fitness programs.",
        "expected": "Gym memberships, personal training, yoga, Pilates, fitness equipment purchases, sports fees, health retreats, outdoor activities, and virtual programs.",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Cross-referenced with PerksPlus health and wellness program details.",
        },
    },
    {
        "id": "Q004",
        "input": "What is the copayment for a specialist visit under the Northwind Standard plan?",
        "context": "The copayment for specialist visits under the Northwind Standard plan is $50.",
        "expected": "$50",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Matched with the cost-sharing section for the Northwind Standard plan.",
        },
    },
    {
        "id": "Q005",
        "input": "Does the Northwind Health Plus plan offer coverage for mental health services?",
        "context": "Northwind Health Plus provides coverage for mental health and substance abuse services.",
        "expected": "Yes",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Validated against the coverage summary for Northwind Health Plus.",
        },
    },
    {
        "id": "Q006",
        "input": "Are acupuncture services covered under the Northwind Health Plus plan?",
        "context": "Northwind Health Plus covers up to 12 medically necessary acupuncture visits per year, with prior approval required.",
        "expected": "Yes, up to 12 visits per year with prior approval.",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Aligned with the acupuncture coverage section.",
        },
    },
    {
        "id": "Q007",
        "input": "What is the reimbursement limit for fitness-related expenses under PerksPlus?",
        "context": "Employees can expense up to $1,000 annually for fitness-related programs under PerksPlus.",
        "expected": "$1,000",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Confirmed from the PerksPlus overview.",
        },
    },
    {
        "id": "Q008",
        "input": "What types of dental services are covered under Northwind Health Plus?",
        "context": "Northwind Health Plus covers dental exams, cleanings, and fillings as part of its vision and dental coverage.",
        "expected": "Dental exams, cleanings, and fillings.",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Verified against the dental services coverage for Northwind Health Plus.",
        },
    },
    {
        "id": "Q009",
        "input": "What is the out-of-pocket maximum for family coverage under Northwind Standard?",
        "context": "The out-of-pocket maximum for family coverage under Northwind Standard is $12,700.",
        "expected": "$12,700",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Cross-checked with Northwind Standard plan details.",
        },
    },
    {
        "id": "Q010",
        "input": "Does the Northwind Health Plus plan protect against balance billing?",
        "context": "Northwind Health Plus protects members from balance billing for in-network services.",
        "expected": "Yes, for in-network services.",
        "metadata": {
            "annotation": "good",
            "annotation_note": "Aligned with balance billing protection details.",
        },
    },
]

dataset_rows = []
for row in eval_dataset:
    dataset_rows.append({
        "input": row["input"],
        "context": row["context"],
        "expected": row["expected"],
        "metadata": {
            "annotation": row["metadata"]["annotation"],
            "annotation_note": row["metadata"]["annotation_note"],
        }
    })

rag_dataset = quotient_client.datasets.create(
    name="Northwind Health Evaluation",
    description="Evaluation dataset for RAG with Northwind Health data.",
    rows=dataset_rows
)

print("\nCreated dataset in QuotientAI:")
print(f"Dataset ID: {rag_dataset.id}")
print(f"Name: {rag_dataset.name}")
print(f"Number of rows: {len(rag_dataset.rows)}")


Created dataset in QuotientAI:
Dataset ID: ea38ea5c-2ed5-427c-8568-73a30d3d158f
Name: Northwind Health Evaluation
Number of rows: 10


## 7. Create a Prompt in Quotient

In [26]:
prompt = quotient_client.prompts.create(
    name="my-northwind-prompt",
    system_prompt="You are a helpful assistant with knowledge of Contoso HR. Answer accurately based on context.",
    user_prompt="User query: {input}\nContext: {context}\nPlease provide a concise, accurate answer."
)

print("\nCreated Prompt in QuotientAI:")
print(f"Prompt ID: {prompt.id}")
print(f"Prompt Name: {prompt.name}")
print(f"System Prompt: {prompt.system_prompt}")
print(f"User Prompt: {prompt.user_prompt}")


Created Prompt in QuotientAI:
Prompt ID: 9757aa01-2508-439f-9a84-d33d8700ced0
Prompt Name: my-northwind-prompt
System Prompt: You are a helpful assistant with knowledge of Contoso HR. Answer accurately based on context.
User Prompt: User query: {input}
Context: {context}
Please provide a concise, accurate answer.
